In [613]:
from math import log

In [677]:
import pandas as pd
import numpy as np
from pprint import pprint
data = pd.DataFrame({"toothed":["True","True","True","False","True","True","True","True","True","False"],"breathes":["True","True","True","True","True","True","False","True","True","True"],"legs":["True","True","False","True","True","True","False","False","True","True"],"species":["Mammal","Mammal","Reptile","Mammal","Mammal","Mammal","Reptile","Reptile","Mammal","Reptile"]})
# data = pd.read_csv("tennis.csv")
# # data = pd.read_csv("hd.csv")
print(data)
features= data.columns[:-1]
print(features)
target = data.columns[features.size]
print(target)
columns=data.columns
# data

  toothed breathes   legs  species
0    True     True   True   Mammal
1    True     True   True   Mammal
2    True     True  False  Reptile
3   False     True   True   Mammal
4    True     True   True   Mammal
5    True     True   True   Mammal
6    True    False  False  Reptile
7    True     True  False  Reptile
8    True     True   True   Mammal
9   False     True   True  Reptile
Index(['toothed', 'breathes', 'legs'], dtype='object')
species


In [678]:
def entropy(column):
    elements,counts = np.unique(column,return_counts = True)
    entropy = 0
    for i in range(elements.size):
        entropy += counts[i]/np.sum(counts)*log(counts[i]/np.sum(counts),2)
    entropy= -entropy
    return entropy

In [679]:
def InfoGain(data,attribute_name):
    attribute= data.get(attribute_name)
    total_entropy = entropy(data.get(target))
    elements,counts = np.unique(attribute,return_counts = True)
    split_columns={}
    for i in range(elements.size):
        split_columns[i] = data[attribute == elements[i]]  
    weighted_Entropy = 0
    for i in range(elements.size):
        weighted_Entropy += (counts[i]/np.sum(counts))*entropy(split_columns[i].get(target))
    Info_gain = total_entropy - weighted_Entropy 
    return Info_gain

In [680]:
def PLURALITY_VALUE(example):
    elememts, count = np.unique(example[target],return_counts=True)
    selected_index =  np.argmax(count)
#     print(np.argmax(count))
#     print(np.unique(example[target])[np.argmax(count)])
    return np.unique(example[target])[selected_index]

In [681]:
def decision_tree_learning(examples, attributes, parent_examples):
    
# if examples is empty then return PLURALITY_VALUE(parent_examples)
    if examples.size==0 :
        return PLURALITY_VALUE(parent_examples)
    
# else if all examples have same classification then return the classification
    elif np.unique(examples[target]).size < 2:
        return np.unique(examples[target])[0]
    
# else if attributes is empty then return PLURALITY_VALUE(examples)
    elif attributes.size == 0:
        return PLURALITY_VALUE(examples)
    
    else:
        
        InfoGain_values = np.empty((0, attributes.size))
        for feature in attributes:
            InfoGain_values = np.append(InfoGain_values,InfoGain(examples,feature))
        
        selected_column = examples.get(attributes[np.argmax(InfoGain_values)])
        best_attribute = attributes[np.argmax(InfoGain_values)]
        
        tree = {best_attribute:{}}  # here best_attribute = A in pseudoCode
        
        featuresNew = np.empty((0, attributes.size-1))
        for i in range(attributes.size):
            if(attributes[i]!=best_attribute):
                featuresNew = np.append(featuresNew,attributes[i])
                
        elements,counts=np.unique(selected_column,return_counts=True)
        
        for i in range((np.unique(selected_column)).size):
            value = np.unique(selected_column)[i]
            exs = examples.get(examples[best_attribute] == value)
            subtree = decision_tree_learning(exs,featuresNew,PLURALITY_VALUE(examples))
            tree[best_attribute][value] = subtree
        return tree

In [682]:
tree= decision_tree_learning(data,features, None)
pprint(tree)

{'legs': {'False': 'Reptile',
          'True': {'toothed': {'False': {'breathes': {'True': 'Mammal'}},
                               'True': 'Mammal'}}}}
